<a href="https://colab.research.google.com/github/noobmaster-ru/numerical_methods_6sem/blob/main/numerical_methods_task_3_kozlov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вычисление

## Импорт библиотек

In [ ]:
# импорт библиотек
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
import math

## Задание начальных и граничных условий

In [ ]:
# Задание коэффициентов
def p(x):
    if 0 <= x <= np.pi / 3:
        return (x - np.pi / 2) ** 2
    elif np.pi / 3 < x <= 2 * np.pi / 3:
        return x ** 2
    elif 2 * np.pi / 3 < x <= np.pi:
        return 0.1 + (x - np.pi) ** 2

def q(x):
    return 1.0

def f(x):
    if 0 <= x <= np.pi / 2:
        return np.sin(x)
    else:
        return 1 + np.sin(x)


## Генерация сетки

In [ ]:
# Неравномерная сетка с сгущением к концам
def generate_grid(N):
    # xi = np.linspace(0, 1, N+1)
    # x = np.pi * xi**2  # сгущение к нулю
    # return x

    """
    Генерация неравномерной сетки с узлами в точках pi/3 и 2pi/3.
    Сетка состоит из 3 подотрезков: [0, pi/3], [pi/3, 2pi/3], [2pi/3, pi].
    """
    x0 = 0
    x1 = np.pi / 3
    x2 = 2 * np.pi / 3
    x3 = np.pi

    # Делим количество отрезков пропорционально длинам подотрезков
    total_length = x3 - x0
    len1 = x1 - x0
    len2 = x2 - x1
    len3 = x3 - x2

    N1 = int(np.round(N * len1 / total_length))
    N2 = int(np.round(N * len2 / total_length))
    N3 = N - N1 - N2  # остаток

    # Строим сетку на каждом подотрезке, включая точки стыков
    grid1 = np.linspace(x0, x1, N1 + 1, endpoint=False)  # [0, pi/3)
    grid2 = np.linspace(x1, x2, N2 + 1, endpoint=False)  # [pi/3, 2pi/3)
    grid3 = np.linspace(x2, x3, N3 + 1)                  # [2pi/3, pi]

    # Объединяем и возвращаем
    x = np.concatenate([grid1, grid2, grid3])
    # Визуализация
    plt.figure(figsize=(10, 1.5))
    plt.eventplot(x, orientation='horizontal', colors='blue', linewidths=1.2)

    # Добавим вертикальные линии в точках разрыва
    plt.axvline(np.pi/3, color='red', linestyle='--', label=r'$\pi/3$')
    plt.axvline(2*np.pi/3, color='green', linestyle='--', label=r'$2\pi/3$')

    plt.title("Неравномерная сетка с узлами в точках разрыва")
    plt.yticks([])
    plt.xlabel("x")
    plt.legend()
    plt.grid(True, axis='x', linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()
    return x

In [ ]:
# Дискретизация: центральные разности + интегральная форма
def build_matrix_and_rhs(x):
    N = len(x) - 1
    A = np.zeros((N+1, N+1))
    b = np.zeros(N+1)

    # Внутренние узлы
    for i in range(1, N):
        h_minus = x[i] - x[i-1]
        h_plus = x[i+1] - x[i]
        h = h_minus + h_plus

        pL = p((x[i] + x[i-1]) / 2)
        pR = p((x[i+1] + x[i]) / 2)

        A[i, i-1] = -pL / h_minus
        A[i, i] = pL / h_minus + pR / h_plus + q(x[i]) * h / 2
        A[i, i+1] = -pR / h_plus
        b[i] = f(x[i]) * h / 2

    # Граничные условия
    A[0, 0] = 1
    b[0] = 0  # u(0) = 0

    # p(L) u' + u = 1
    hN = x[N] - x[N-1]
    A[N, N-1] = -p((x[N]+x[N-1])/2) / hN
    A[N, N] = p((x[N]+x[N-1])/2) / hN + 1
    b[N] = 1

    return A, b

In [ ]:
# Основной расчет
def solve_bvp(N):
    # вот здесь надо сетку строить неравномерную - три отрезка - на каждом отрезке, допустим, N точек , правый конец не включается
    # причем надо попасть узлом в точку разрыва - чтобы точка разрыва была в каком-то узле
    x = generate_grid(N)

    A, b = build_matrix_and_rhs(x)
    u = solve(A, b) # надо заменить на численным метод
    return x, u

In [ ]:
# Демонстрация
N = 100
x, u = solve_bvp(N)

fig = plt.figure(figsize=(16, 8))
plt.plot(x, u, label=f"N={N}", color="blue", linewidth = 1.5)

plt.xlabel("x",labelpad = 10, fontsize = 20, fontweight = 'bold')
plt.ylabel("u(x)",rotation = 0 ,labelpad = 10, fontsize = 20, fontweight = 'bold')

plt.title("Численное решение краевой задачи", fontsize=20)

plt.grid(True, color='gray' , which='both', linestyle='--', linewidth=0.6, alpha=0.9)
plt.minorticks_on()
plt.grid(True, which='minor', linestyle=':', linewidth=0.6, alpha=0.6)


plt.legend()
plt.show()

# Оценка сходимости методом Рунге

In [ ]:
def linear_interp(N, u2_vals, r):
    """Линейная интерполяция y(x) по массивам x_vals, y_vals"""
    result = []
    for i in range(N):
      result.append(u2_vals[i*r])
    return result

In [ ]:
def max_norm_error(u1, u2_interp):
    """Вычисление нормы max|u1 - u2_interp|"""
    max_err = 0.0
    for a, b in zip(u1, u2_interp):
        err = abs(a - b)
        if err > max_err:
            max_err = err
    return max_err

In [ ]:
def mse_norm(u):
  mse_norm = 0.0
  for i in range(len(u)):
    mse_norm += u[i]*u[i]
  mse_norm = np.sqrt(mse_norm)
  return mse_norm

In [ ]:
# Метод Рунге для оценки порядка
def runge_convergence(N_coarse):
  r = 2
  x1, u1 = solve_bvp(N_coarse)
  x2, u2 = solve_bvp(N_coarse * r)
  x4, u4 = solve_bvp(N_coarse * r * r)


  # вот здесь неправильно наверно считаю - надо вместо ошибок брать численное решение и искать норму между численным решение и истинным,
  #         но вот как найти истинное решение пока не понял

  # Интерполяция на мелкой сетке к крупной
  u12_interp = linear_interp(N_coarse, u2, r)
  u14_interp = linear_interp(N_coarse, u4, r*r)

  mse_norm_n = mse_norm(u1)
  mse_norm_2n = mse_norm(u2)
  mse_norm_4n = mse_norm(u4)
  rate = np.log2((mse_norm_n - mse_norm_2n) / (mse_norm_2n - mse_norm_4n))

  # error_n = max_norm_error(u1, u2_interp)
  # error_2n = max_norm_error(u1, u2_interp)
  # rate = np.log2( (errors[i-1] - errors[i]) / (errors[i] - errors[i+1]))
  return  rate

In [ ]:
# Оценка сходимости
Ns = [100, 200, 400, 800]
for N in Ns:
  rate = runge_convergence(N)
  print(f"N={N}: порядок ≈ {rate:.2f}")
  # print(f"N={N}: error_n = {error_n:.6f}, порядок ≈ {rate:.2f}")

# print(errors)
# for i in range(1,len(errors)-1):
  # rate = np.log2( (errors[i-1] - errors[i]) / (errors[i] - errors[i+1]))
  # print(errors[i-1] - errors[i])
  # print(errors[i] - errors[i+1])